In [95]:
import os
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from substitutions import appos
from nltk.stem import PorterStemmer, WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import string
import numpy as np
import contractions
import copy

[nltk_data] Downloading package stopwords to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Samyak
[nltk_data]     Jain\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [96]:
data_dir = '../Humor,Hist,Media,Food'
file_names = os.listdir(data_dir)
file_paths = [(data_dir + '/' + fname) for fname in file_names]
docID_to_doc = {}
for i in range(len(file_names)):
    docID_to_doc[i] = file_names[i]

In [97]:
x = contractions.fix("We're ain't no don't he's going to.")
print(x)

We are are not no do not he is going to.


In [101]:
# stop_words = set(stopwords.words('english'))
# stemmer = PorterStemmer()
# lemmatizer = WordNetLemmatizer()
def read_files(fpaths):
    file_tokens = []
    for fpath in fpaths:
        f = open(fpath, 'r', encoding='utf-8', errors='replace')
        ftxt_unprocessed = f.read()
        # print(ftxt_unprocessed)
        ftoks = preprocess_file(ftxt_unprocessed)
        file_tokens.append(ftoks)
    return file_tokens

def isValidTok(tok):
    if((tok not in string.punctuation) and (tok.isnumeric() == False) and (sum([0 if ch in string.punctuation else 1 for ch in tok]) >= 1)):
        return True
    return False 
def check_alnum(tok):
    tok = ''.join(x for x in tok if x.isalnum() == True)
    return tok
def remove_punct(tok):
    punctuations = string.punctuation
    tok = ''.join(x for x in tok if x not in punctuations)
    return tok

def remove_space(tok):
    tok = ''.join(x for x in tok if x != ' ')
    return tok

def preprocess_file(file_text):
    '''
        Preprocess the file text
    '''
    #converting text to lowercase
    text = file_text.lower()
    
    # print(f"Original tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")

    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = set(all_toks)

    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")
    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")
    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != "" and ctok.isnumeric() == False  and len(ctok) > 1):
            cleaned_toks.append(ctok)
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" and len(stok) > 1 and (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    final_tokens = [tok for tok in cleaned_toks]

    return final_tokens

# def preprocess_file(file_text):
#     '''
#         Preprocess the file text
#     '''
    
#     #converting to lower case
#     text = file_text.lower()

#     #fix the contractions
#     text = contractions.fix(text)



#     #word tokenization
#     all_tokens = word_tokenize(text)
#     # print(f"All toks {all_tokens}")
#     all_unique_tokens = set(all_tokens)

#     # print(all_unique_tokens)

#     lemmatized_toks = set()
#     for unq_tok in all_unique_tokens:
#         stemmed_tok = lemmatizer.lemmatize(unq_tok)
#         lemmatized_toks.add(stemmed_tok)
#     # print(lemmatized_toks)
    
#     tokens = list(lemmatized_toks - stop_words)
    

#     # ps = PorterStemmer()
#     valid_toks = []
#     for tok in tokens:
#         if(isValidTok(tok) == True):
#             valid_toks.append(tok)
#     return valid_toks
#     # print(final_tokens)

def preprocess_query(query_text):

    text = query_text.lower()
    
    # print(f"Original Query Tokens : \n{word_tokenize(text)}\n")
    # print("\n-------------------------\n")
    
    #Fixing the contractions
    text = contractions.fix(text)

    #omitting all non-alphanumeric characters
    # text = ''.join([x if (x.isalnum() == True or x == " ") else " " for x in text ])

    #word tokenization
    all_toks = word_tokenize(text)
    all_unique_toks = []
    for tok in all_toks:
        if(tok not in all_unique_toks):
            all_unique_toks.append(tok)

    all_unique_toks = [check_alnum(x) for x in all_unique_toks]

    # print(f"All unique tokens :\n{all_unique_toks}\n")
    # print("\n-------------------------\n")
    #removing stopwords
    stop_words = list(set(stopwords.words('english')))
    file_toks = [tok for tok in all_unique_toks if tok not in stop_words]
    # print(f"After removing stopwords :\n{file_toks}\n")
    # print("\n-------------------------\n")
    #removing punctations if any remain
    toks_no_punct = []
    for tok in file_toks:
        ctok = remove_punct(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != ""):
            toks_no_punct.append(ctok)
    # print(f"After removing puncts :\n{toks_no_punct}\n")
    # print("\n-------------------------\n")

    #removing spaces in any remain & check if it is a number
    cleaned_toks = []
    for tok in toks_no_punct:
        ctok = remove_space(tok)
        # print(f"-> before removal : {tok} || After removal : {ctok}")
        if(ctok != "" and ctok.isnumeric() == False and len(ctok) > 1):
            cleaned_toks.append(ctok)
    # print(f"After removing spaces and removing numbers :\n{cleaned_toks}\n")
    # print("\n-------------------------\n")
    #stemming tokens
    # stemmer = PorterStemmer()
    # stemmed_toks = set()

    # for tok in cleaned_toks:
    #     stok = stemmer.stem(tok)
    #     if(stok != "" and len(stok) > 1 and (stok not in stop_words)): #consider non-blank tokens and the ones which have length greater than 1
    #         stemmed_toks.add(stok)
    # print(f"After stemming and removing single letters :\n{stemmed_toks}\n")
    # print("\n-------------------------\n")
    final_tokens = [tok for tok in cleaned_toks]

    return final_tokens



    

In [102]:
file_toks = read_files(file_paths)

In [103]:
def create_inverted_index(file_toks):
    inv_index_postings = {}
    inv_index_frequency = {}
    for i in range(len(file_toks)):
        for tok in file_toks[i]:
            if(tok not in inv_index_postings.keys()):
                inv_index_postings[tok] = [i]
                inv_index_frequency[tok] = 1
            else:
                inv_index_postings[tok].append(i)
                inv_index_frequency[tok] += 1
    inv_index_postings = dict(sorted(inv_index_postings.items()))
    terms_list = inv_index_postings.keys()
    for word in terms_list:
        inv_index_postings[word].sort()
    return inv_index_postings, inv_index_frequency

In [104]:
def getDocsFromID(file_names, doc_IDs):
    doc_names = []
    for doc_ID in doc_IDs:
        doc_names.append(file_names[doc_ID])
    return doc_names

In [105]:
def check_equal_arrays(arr1, arr2):

    if(len(arr1) != len(arr2)):
        return False
    
    arr1 = sorted(arr1)
    arr2 = sorted(arr2)
    for i in range(len(arr1)):
        if(arr1[i] != arr2[i]):
            return False
    return True

In [106]:
def retreive_postings(inv_index_postings, term):
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return []
    else:
        posting = inv_index_postings[term]
        return posting

In [107]:
def retreive_frequency(inv_index_frequency, term):
    terms_list = inv_index_frequency.keys()
    if(term not in terms_list):
        return 0
    else:
        freq = inv_index_frequency[term]
        return freq

In [108]:
def retreive_term_info(inv_index_postings, inv_index_frequency, term):
    terms_list = inv_index_postings.keys()
    if(term not in terms_list):
        return [], 0
    else:
        posting = inv_index_postings[term]
        freq = inv_index_frequency[term]
        return posting, freq

In [109]:
def query_AND(posting1, posting2, verbose=False):


    if(posting1 == [] or posting2 == []):
        return 0, 0, []

    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

        if(posting1[ptr1] == posting2[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(file_names, answer_docID)

    if(verbose==True):
        print(f"AND Query: \nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) & set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")

    return num_docs_retreived, num_comparisons, doc_names_retreived

def query_OR(posting1, posting2, verbose=False):
    

    if((posting1 == []) and (posting2 == [])):
        return 0, 0, []
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = posting2
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0

        while(ptr1 < len(posting1) and ptr2 < len(posting2)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

            if(posting1[ptr1] == posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2)):
            answer_docID.append(posting2[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(file_names, answer_docID)

        if(verbose==True):
            print(f"Query OR\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        # temp_verification = list(set(posting1) | set(posting2))
        # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
        return num_docs_retreived, num_comparisons, doc_names_retreived

def perform_NOT(posting):

    all_docIDs = [docID for docID in range(len(file_names))]
    if(posting == []):
        return all_docIDs
    for docID in posting:
        all_docIDs.remove(docID)

    return all_docIDs

def query_AND_NOT(posting1, posting2, verbose=False):

    if((posting1 == [])):
        return 0, 0, []
    
    posting2_NOT = perform_NOT(posting2)
    ptr1 = 0
    ptr2 = 0
    answer_docID = []

    num_comparisons = 0

    while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
        num_comparisons += 1
        # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")
        if(posting1[ptr1] == posting2_NOT[ptr2]):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
            ptr2 += 1
        elif(posting1[ptr1] < posting2_NOT[ptr2]):
            ptr1 += 1
        else:
            ptr2 += 1

    num_docs_retreived = len(answer_docID)
    doc_names_retreived = getDocsFromID(file_names, answer_docID)

    if(verbose==True):
        print(f"Query AND NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
    
    # temp_verification = list(set(posting1) | set(posting2))
    # print(f"Verification: {check_equal_arrays(temp_verification, answer_docID)}")
    return num_docs_retreived, num_comparisons, doc_names_retreived

def query_OR_NOT(posting1, posting2, verbose=False):
    
    if((posting1 == []) and (posting2 == [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 == []) and (posting2 != [])):
        ans_docs = perform_NOT(posting2)
        return len(ans_docs), 0, ans_docs
    elif((posting1 != []) and (posting2 == [])):
        ans_docs = posting1
        return len(ans_docs), 0, ans_docs
    else:
        
        posting2_NOT = perform_NOT(posting2)
        ptr1 = 0
        ptr2 = 0
        answer_docID = []

        num_comparisons = 0

        while(ptr1 < len(posting1) and ptr2 < len(posting2_NOT)):
            num_comparisons += 1
            # print(f"1 : {posting1[ptr1]} , 2: {posting2[ptr2]}")

            if(posting1[ptr1] == posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
                ptr2 += 1
            elif(posting1[ptr1] < posting2_NOT[ptr2]):
                answer_docID.append(posting1[ptr1])
                ptr1 += 1
            else:
                answer_docID.append(posting2_NOT[ptr2])
                ptr2 += 1
        while(ptr1 < len(posting1)):
            answer_docID.append(posting1[ptr1])
            ptr1 += 1
        while(ptr2 < len(posting2_NOT)):
            answer_docID.append(posting2_NOT[ptr2])
            ptr2 += 1


        num_docs_retreived = len(answer_docID)
        doc_names_retreived = getDocsFromID(file_names, answer_docID)

        if(verbose==True):
            print(f"Query OR NOT\nNo. of documents retreived: {num_docs_retreived}\nMinimum number of comparisons: {num_comparisons}\nNames of retreived documents: {doc_names_retreived}")
        
        return num_docs_retreived, num_comparisons, doc_names_retreived

In [110]:
inverted_index_postings, inverted_index_frequency = create_inverted_index(file_toks)

In [111]:
getDocsFromID(file_names, [1])

['a-team']

In [116]:
postings1 = retreive_postings(inverted_index_postings, 'piano')
postings2 = retreive_postings(inverted_index_postings, 'hand')
num_docs_AND, min_cmps_AND, doc_names_AND = query_AND_NOT(postings1, postings2, verbose=True)

Query AND NOT
No. of documents retreived: 16
Minimum number of comparisons: 848
Names of retreived documents: ['aids.txt', 'bbc_vide.cat', 'drunk.txt', 'english', 'final-ex.txt', 'finalexm.hum', 'gd_tznew.txt', 'hoosier.txt', 'insanity.hum', 'lawskool.txt', 'lawyer.jok', 'quotes.jok', 'smackjok.hum', 'smartass.txt', 'wagon.hum', 'woodbugs.txt']


In [ ]:
def perform_query(posting1, posting2, op):
    if(op == "AND"):
        return query_AND(posting1, posting2)
    elif(op == "OR"):
        return query_OR(posting1, posting2)
    elif(op == "AND NOT"):
        return query_AND_NOT(posting1, posting2)
    elif(op == "OR NOT"):
        return query_OR_NOT(posting1, posting2)
    else:
        print("INVALID OPERATOR")
        return []

def process_query(query_text, ops):
    query_tokens = preprocess_query(query_text)
    print(f"Query tokens : {query_tokens}\n")
    print(f"Operators : ")
    n_qtoks = len(query_tokens)
    n_ops = n_qtoks - 1
    # query_operato = copy.deepcopy(ops)
    if(len(ops) > n_ops):
        print(f"Insufficient tokens in the query\n")
        return []
    elif(len(ops) < n_ops):
        print(f"Insufficient operators")
        return []
    else:
        if(n_ops == 1):
            p1 = retreive_postings(query_tokens[0])
            p2 = retreive_postings(query_tokens[1])
            return perform_query(p1, p2, ops[0])
        else:
            p1 = retreive_postings(query_tokens[0])
            p2 = retreive_postings(query_tokens[1])
            ans = perform_query(p1, p2, ops[0])
            qtok_ptr = 2
            for i in range(1, len(ops)):
                posting_tok = retreive_postings(query_tokens[qtok_ptr])
                ans = perform_query(ans, posting_tok, ops[i])
                qtok_ptr += 1
            return ans